In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
stop_words = stopwords.words('english')


In [ ]:
# Training data
train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
# Testing data
test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
# sample 
sample = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')


In [ ]:
#train['comment_text'].iloc[125698]


In [ ]:
pd.set_option('display.max_colwidth', None)
train['comment_text'].sample(5)


In [ ]:
# problems in data
#[User:Cirt]]
#200.83.101.199
#(Romania)
#hi moron
# URLs


In [ ]:
# Label column names
labels = list(train.columns[2:])
labels


In [ ]:
label = train[['toxic', 'severe_toxic' , 'obscene' , 'threat' , 'insult' , 'identity_hate']]
print(label.head())


In [ ]:
ct1,ct2 = 0,0
for i in range(label.shape[0]):
    ct = np.count_nonzero(label.iloc[i])
    if ct :
        ct1 = ct1+1
    if ct>1 :
        ct2 = ct2+1
print(ct1)
print(ct2)


In [ ]:
x = [len(train['comment_text'][i]) for i in range(train['comment_text'].shape[0])]

print('average length of comment: {:.3f}'.format(sum(x)/len(x)) )
bins = [1,200,400,600,800,1000,1200]
plt.hist(x, bins=bins)
plt.xlabel('Length of comments')
plt.ylabel('Number of comments')       
plt.axis([0, 1200, 0, 90000])
plt.grid(True)
plt.show()


In [ ]:
label.head(1)


In [ ]:
y = np.zeros(label.shape)
for ix in range(train['comment_text'].shape[0]):
    l = len(train['comment_text'][ix])
    if label['toxic'].iloc[ix] :
        y[ix][0] = l
    if label['severe_toxic'].iloc[ix] :
        y[ix][1] = l
    if label['obscene'].iloc[ix] :
        y[ix][2] = l
    if label['threat'].iloc[ix] :
        y[ix][3] = l
    if label['insult'].iloc[ix] :
        y[ix][4] = l
    if label['identity_hate'].iloc[ix] :
        y[ix][5] = l

labelsplt = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
color = ['red','green','blue','yellow','orange','chartreuse']  
plt.figure(figsize=(20,20))
plt.hist(y,bins = bins,label = labelsplt,color = color)
plt.axis([0, 1200, 0, 8000])
plt.xlabel('Length of comments')
plt.ylabel('Number of comments') 
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.barh(labelsplt,train[labels].sum(axis = 0),color = color)


In [ ]:
# get value counts for each class
train[labels].sum(axis = 0)


In [ ]:
# get number of comments that have been classified (toxic)
# comments_classified = sum(train[labels].sum(axis = 1) !=0)
# comments_classified


In [ ]:
# comments_not_classified = comments - comments_classified
# print(f"there are {comments_not_classified} not toxic comments in the dataset")


In [ ]:
# # adding not toxic column
# condition = train[labels].sum(axis = 1) == 0
# train['not_toxic'] = np.where(condition, 1,0)


In [ ]:
# # checking if there is an undefined toxicity type
# sum(train[labels].sum(axis = 1) == 1)


In [ ]:
# # adding toxicity undefined column
# condition = train[labels].sum(axis = 1) == 1
# train['undefined_toxic'] = np.where(condition, 1,0) 


In [ ]:
# # checking if there is a comment classified with a toxic type without classifying it as toxic
# train['toxicity_type_defined'] = train[['insult','obscene','identity_hate','threat']].max(axis=1)
# condition = (train['toxicity_type_defined']==1) & (train['toxic']==0)
# train['soft_toxic'] = np.where(condition, 1,0)
# train.drop(['toxicity_type_defined'], axis = 1, inplace = True)


In [ ]:
# # adding new labels
# labels.extend(['not_toxic', 'undefined_toxic', 'soft_toxic'])


In [ ]:
# label_counts = train[labels].sum()
# plt.figure(figsize=(20, 10))
# ax = sns.barplot(x=label_counts.index, y=label_counts.values)
# ax.set_yscale("log")
# ax.tick_params(labelsize=15)


In [ ]:
# heatmap_data = train[labels]
# plt.figure(figsize=(10, 10))
# ax = sns.heatmap(heatmap_data.corr(), cmap='coolwarm', annot=True)
# ax.tick_params(labelsize=10)


In [ ]:
# # Confirm that all severly toxic comments (n=1595) are toxic:
# train.loc[train['severe_toxic']==1,'toxic'].sum()


In [ ]:
def to_lowercase(text):
    return text.lower()

# Remove website links
def remove_links(text):
    template = re.compile(r'https?://\S+|www\.\S+') 
    text = template.sub(r'', text)
    return text

# Remove HTML tags
def remove_html(text):
    template = re.compile(r'<[^>]*>') 
    text = template.sub(r'', text)
    return text

def text2words(text):
      return word_tokenize(text)

# Remove stopwords
def remove_stopwords(words, stop_words):
    return [word for word in words if word not in stop_words]

# Remove none ascii characters
def remove_non_ascii(text):
    template = re.compile(r'[^\x00-\x7E]+') 
    text = template.sub(r'', text)
    return text

# Replace none printable characters
def remove_non_printable(text):
    template = re.compile(r'[\x00-\x0F]+') 
    text = template.sub(r' ', text)
    return text

# Remove special characters
def remove_special_chars(text):
        text = re.sub("'s", '', text)
        template = re.compile('["#$%&\'()\*\+-/:;<=>@\[\]\\\\^_`{|}~]') 
        text = template.sub(r' ', text)
        return text

# Replace multiple punctuation 
def replace_multiplt_punc(text):
        text = re.sub('[.!?]{2,}', '.', text)
        text = re.sub(',+', ',', text) 
        return text


def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


# Remove numbers
def remove_numbers(text):
        text = re.sub('\d+', ' ', text)
        return text

def handle_spaces(text):
    # Remove extra spaces
    text = re.sub('\s+', ' ', text)
    
    # Remove spaces at the beginning and at the end of string
    text = text.strip() 
    
    return text

def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ([lemmatizer.lemmatize(word, pos='v') for word in words])

def remove_pattern(text): 
    # remove hi moron 
    text= re.sub(r'(hi)(.*)\1', r'\1', text)
    # remove duplicate words
    text= re.sub(r"\b(\w+)(?:\W+\1\b)+",r'\1', text,flags=re.IGNORECASE)
    # remove [User:Cirt]] 
    text= re.sub(r"\[.*?\]", ' ', text)
    # remove \n\n
    text= re.sub(r"\n", ' ', text)
    return text

def clean_text( text):
    text = remove_pattern(text)
    text = remove_links(text)
    text = remove_html(text)
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_non_printable(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = handle_spaces(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    #words = stem_words(words) #either stem or lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ' '.join(words)


In [ ]:
print(train['comment_text'].iloc[2])
print('-------------------')
sample = clean_text(train['comment_text'].iloc[2])
sample


In [ ]:
def return_tweets(df):
    texts=[]
    for index, item in df.drop(df.columns.difference(['comment_text']), axis=1).iterrows():
        message = item["comment_text"]
        texts.append(str(message))
    return texts


In [ ]:
train_texts = return_tweets(train)
test_texts = return_tweets(test)


In [ ]:
def clean_corpus(corpus):
    return [clean_text(t) for t in corpus]
train_texts = clean_corpus(train_texts)
test_texts = clean_corpus(test_texts)


In [ ]:
train_texts[:5]


In [ ]:
x = [len(train_texts[i]) for i in range(len(train_texts))]

print('average length of comment: {:.3f}'.format(sum(x)/len(x)) )
bins = [1,200,400,600,800,1000,1200]
plt.hist(x, bins=bins)
plt.xlabel('Length of comments')
plt.ylabel('Number of comments')       
plt.axis([0, 1200, 0, 90000])
plt.grid(True)
plt.show()


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer  
tok = Tokenizer(num_words=1000, oov_token='UNK')
#tok = Tokenizer(oov_token='UNK')
tok.fit_on_texts(train_texts + test_texts)
# Extract binary BoW features
x_train = tok.texts_to_matrix(train_texts, mode='tfidf')
x_test = tok.texts_to_matrix(test_texts, mode='tfidf')


In [ ]:
y_train = np.asarray(label.values).astype('float32')


In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)


In [ ]:
from sklearn.model_selection import train_test_split
tr_X, val_X, tr_y, val_y = train_test_split(x_train, y_train, train_size=0.95, random_state=42)


In [ ]:
print(tr_X.shape)
print(tr_y.shape)
print(val_X.shape)
print(val_y.shape)


In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(1000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(6, activation='sigmoid'))

model.compile(optimizer=optimizers.Adam(),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

history = model.fit(tr_X,
                    tr_y,
                    epochs=20,
                    batch_size=512,
                    validation_data=(val_X, val_y))


In [ ]:
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()


In [ ]:
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()

plt.show()


In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(1000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(6, activation='sigmoid'))

model.compile(optimizer=optimizers.Adam(),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

history = model.fit(tr_X,
                    tr_y,
                    epochs=9,
                    batch_size=512,
                    validation_data=(val_X, val_y))
# compute roc_auc score  
from sklearn.metrics import roc_auc_score
y_pred = model.predict(val_X)
roc_auc_score(val_y, y_pred)


In [ ]:
#How to create a submission csv file for Kaggle
df_sample = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip")
y_pred = model.predict(x_test)
df_sample[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
df_sample.to_csv('submission.csv', index=False)
